In [1]:
# importing necessary libaries
import pandas as pd
from pathlib import Path

# creating file paths
school_csv = Path("Resources/schools_complete.csv")
student_csv = Path("Resources/students_complete.csv")

# creating DataFrames from .csv files
school = pd.read_csv(school_csv)
student = pd.read_csv(student_csv)

# combining DataFrames to a single DataFrame
combined = pd.merge(school, student, how="left", on=["school_name", "school_name"]).reset_index(drop=True)

# printing combined DataFrame
combined

,School ID,school_name,type,size,budget,Student ID,student_name,gender,year,reading_score,maths_score
0,0,Huang High School,Government,2917,1910635,0,Paul Bradley,M,9,96,94
1,0,Huang High School,Government,2917,1910635,1,Victor Smith,M,12,90,43
2,0,Huang High School,Government,2917,1910635,2,Kevin Rodriguez,M,12,41,76
3,0,Huang High School,Government,2917,1910635,3,Richard Scott,M,12,89,86
4,0,Huang High School,Government,2917,1910635,4,Bonnie Ray,F,9,87,69
...,...,...,...,...,...,...,...,...,...,...,...
39165,14,Thomas High School,Independent,1635,1043130,39165,Donna Howard,F,12,51,48
39166,14,Thomas High School,Independent,1635,1043130,39166,Dawn Bell,F,10,81,89
39167,14,Thomas High School,Independent,1635,1043130,39167,Rebecca Tanner,F,9,99,99
39168,14,Thomas High School,Independent,1635,1043130,39168,Desiree Kidd,F,10,72,77


## **LGA SUMMARY**

In [2]:
# total count of schools
t_schools = combined["school_name"].nunique()

# total count of students
t_students = len(combined["student_name"])

# total budget
t_budget = combined["budget"].unique().sum()

# avg maths score for entire DataFrame
avg_maths = combined.maths_score.mean()

# avg reading score for entire DataFrame
avg_reading = combined.reading_score.mean()

# percentage of students with passing maths score (50 or higher)
maths_passed = combined[combined["maths_score"] >= 50].reset_index(drop=True)
maths_passed_count = len(maths_passed)
maths_passed_percentage = maths_passed_count / t_students * 100

# percentage of students with passing reading score (50 or higher)
reading_passed = combined[combined["reading_score"] >= 50].reset_index(drop=True)
reading_passed_count = len(reading_passed)
reading_passed_percentage = reading_passed_count / t_students * 100

# percentage of students who passed maths and reading
overall_passed = combined[(combined["maths_score"] >= 50) & (combined["reading_score"] >= 50)].reset_index(drop=True)
overall_passed_count = len(overall_passed)
overall_passed_percentage = overall_passed_count / t_students * 100

# creating LGA summary DataFrame
area_summary = pd.DataFrame([{"Total schools": t_schools,
                   "Total Students": t_students,
                   "Total Budget": t_budget,
                   "Average Maths Score": avg_maths,
                   "Average Reading Score": avg_reading,
                   "% Passing Maths": maths_passed_percentage,
                   "% Passing Reading": reading_passed_percentage,
                   "% Overall Passing": overall_passed_percentage}])

# formatting total budget to currency standard
area_summary["Total Budget"] = area_summary["Total Budget"].map("${:,.2f}".format)

area_summary

,Total schools,Total Students,Total Budget,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
0,15,39170,"$24,649,428.00",70.338192,69.980138,86.078632,84.426857,72.808272


## **SCHOOL SUMMARY**

In [3]:
# creating school summary DataFrame using existing school .csv file, and then renaming columns and setting index to school name
school_updated = pd.read_csv(school_csv).sort_values("school_name").drop("School ID", axis=1)

school_updated = school_updated.rename(columns={"school_name": "School Name", "type": "School Type", "size": "Total Students", "budget": "Total School Budget"})

school_updated = school_updated.set_index("School Name")

# adding new datapoints
# per student budget
school_updated["Per Student Budget"] = school_updated["Total School Budget"] / school_updated["Total Students"]

# average maths score
school_updated["Average Maths Score"] = combined.groupby("school_name")["maths_score"].mean()

# average reading score
school_updated["Average Reading Score"] = combined.groupby("school_name")["reading_score"].mean()

# % passing maths
school_maths_passed = maths_passed.groupby("school_name")["maths_score"].count()
school_updated["% Passing Maths"] = school_maths_passed / school_updated["Total Students"] * 100

# % passing reading
school_reading_passed = reading_passed.groupby("school_name")["reading_score"].count()
school_updated["% Passing Reading"] = school_reading_passed / school_updated["Total Students"] * 100

# % overall passing
school_overall_passed = overall_passed.groupby("school_name")["student_name"].count()
school_updated["% Overall Passing"] = school_overall_passed / school_updated["Total Students"] * 100

# duplicating school_updated DataFrame as unformatted DataFrame will be used in later stages of homework
per_school_summary = school_updated.copy()

# working with school_updated_formatted from remainder of section of homework
# formatting school budget and per student budget to currency standard
per_school_summary["Total School Budget"] = per_school_summary["Total School Budget"].map("${:,.2f}".format)
per_school_summary["Per Student Budget"] = per_school_summary["Per Student Budget"].map("${:,.2f}".format)

per_school_summary

,School Type,Total Students,Total School Budget,Per Student Budget,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
School Name,,,,,,,,,
Bailey High School,Government,4976,"$3,124,928.00",$628.00,72.352894,71.008842,91.639871,87.379421,80.084405
Cabrera High School,Independent,1858,"$1,081,356.00",$582.00,71.657158,71.359526,90.850377,89.074273,80.785791
Figueroa High School,Government,2949,"$1,884,411.00",$639.00,68.698542,69.077993,81.654798,82.807731,67.650051
Ford High School,Government,2739,"$1,763,916.00",$644.00,69.091274,69.572472,82.438846,82.219788,67.469880
Griffin High School,Independent,1468,"$917,500.00",$625.00,71.788147,71.245232,91.212534,88.487738,81.335150
Hernandez High School,Government,4635,"$3,022,020.00",$652.00,68.874865,69.186408,80.949299,81.877023,66.364617
Holden High School,Independent,427,"$248,087.00",$581.00,72.583138,71.660422,89.929742,88.524590,78.922717
Huang High School,Government,2917,"$1,910,635.00",$655.00,68.935207,68.910525,81.693521,81.453548,66.712376
Johnson High School,Government,4761,"$3,094,650.00",$650.00,68.843100,69.039277,82.062592,81.978576,67.191766


## TOP PERFORMING SCHOOLS (BY % OVERALL PASSING)

In [4]:
# sorting schools_updated in descending order based on '% Overall Passing'
top_schools = school_updated.sort_values("% Overall Passing", ascending=False)

top_schools.head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
School Name,,,,,,,,,
Griffin High School,Independent,1468,917500,625.0,71.788147,71.245232,91.212534,88.487738,81.335150
Cabrera High School,Independent,1858,1081356,582.0,71.657158,71.359526,90.850377,89.074273,80.785791
Bailey High School,Government,4976,3124928,628.0,72.352894,71.008842,91.639871,87.379421,80.084405
Wright High School,Independent,1800,1049400,583.0,72.047222,70.969444,91.777778,86.666667,79.722222
Rodriguez High School,Government,3999,2547363,637.0,72.047762,70.935984,90.797699,87.396849,79.419855


## BOTTOM PERFORMING SCHOOLS (BY % OVERALL PASSING)

In [5]:
# sorting schools_updated in ascending order based on '% Overall Passing'
bottom_schools = school_updated.sort_values("% Overall Passing")

bottom_schools.head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
School Name,,,,,,,,,
Hernandez High School,Government,4635,3022020,652.0,68.874865,69.186408,80.949299,81.877023,66.364617
Huang High School,Government,2917,1910635,655.0,68.935207,68.910525,81.693521,81.453548,66.712376
Johnson High School,Government,4761,3094650,650.0,68.843100,69.039277,82.062592,81.978576,67.191766
Wilson High School,Independent,2283,1319574,578.0,69.170828,68.876916,82.785808,81.296540,67.455103
Ford High School,Government,2739,1763916,644.0,69.091274,69.572472,82.438846,82.219788,67.469880


## MATHS SCORES BY YEAR

In [6]:
# creating Series of maths score mean of each school, and then converting it to a DataFrame to pivot
maths_year = combined.groupby(["school_name", "year"])["maths_score"].mean().to_frame()

# resetting index so that 'school_name' and 'year' become regular columns in preparation of pivoting 
maths_year.reset_index(inplace=True)

# pivoting and renaming columns
maths_scores_by_year = maths_year.pivot(columns="year", index="school_name", values="maths_score")
maths_scores_by_year.columns = ["Year 9", "Year 10", "Year 11", "Year 12"]
                                                    
maths_scores_by_year

,Year 9,Year 10,Year 11,Year 12
school_name,,,,
Bailey High School,72.493827,71.897498,72.374900,72.675097
Cabrera High School,72.321970,72.437768,71.008299,70.604712
Figueroa High School,68.477804,68.331586,68.811001,69.325282
Ford High School,69.021609,69.387006,69.248862,68.617811
Griffin High School,72.789731,71.093596,71.692521,71.469178
Hernandez High School,68.586831,68.867156,69.154412,68.985075
Holden High School,70.543307,75.105263,71.640777,73.409639
Huang High School,69.081754,68.533246,69.431345,68.639316
Johnson High School,69.469286,67.990220,68.637730,69.287393


## READING SCORE BY YEAR

In [7]:
# creating Series of reading score mean of each year of each school, and then converting it to a DataFrame to pivot
reading_year = combined.groupby(["school_name", "year"])["reading_score"].mean().to_frame()

# resetting index so that 'school_name' and 'year' become regular columns in preparation of pivoting 
reading_year.reset_index(inplace=True)

# pivoting and renaming columns
reading_scores_by_year = reading_year.pivot(columns="year", index="school_name", values="reading_score")
reading_scores_by_year.columns = ["Year 9", "Year 10", "Year 11", "Year 12"]
                                                    
reading_scores_by_year

,Year 9,Year 10,Year 11,Year 12
school_name,,,,
Bailey High School,70.901920,70.848265,70.317346,72.195525
Cabrera High School,71.172348,71.328326,71.201245,71.856021
Figueroa High School,70.261682,67.677588,69.152327,69.082126
Ford High School,69.615846,68.988701,70.735964,68.849722
Griffin High School,72.026895,70.746305,72.385042,69.434932
Hernandez High School,68.477569,70.621842,68.418199,69.244136
Holden High School,71.598425,71.096491,73.310680,70.481928
Huang High School,68.670616,69.516297,68.740638,68.671795
Johnson High School,68.719286,69.295029,69.969115,67.992521


## SCORES BY SCHOOL SPENDING

In [8]:
# binning 'Per Student Budget' into categories
bins = [0, 584, 629, 644, 679]

# creating category names
budget_cat = ["<$589", "$585-630", "$630-645", "$645-600"]

# binning school_updated DataFrame made from previous section of homework
school_updated["Per Student Budget"] = school_updated["Per Student Budget"].astype(int)
school_updated["Spending Ranges (Per Student)"] = pd.cut(school_updated["Per Student Budget"], 
                                                         bins, labels=budget_cat, include_lowest=True)

# Filtering school_updated so that scores_school only has relevant columns
spending_summary = school_updated[["Spending Ranges (Per Student)", "Average Maths Score", "Average Reading Score",
                              "% Passing Maths", "% Passing Reading", "% Overall Passing"]]
spending_summary = spending_summary.groupby("Spending Ranges (Per Student)").mean()

# formatting
spending_summary["Average Maths Score"] = spending_summary["Average Maths Score"].map("{:.2f}".format)
spending_summary["Average Reading Score"] = spending_summary["Average Reading Score"].map("{:.2f}".format)
spending_summary["% Passing Maths"] = spending_summary["% Passing Maths"].map("{:.2f}".format)
spending_summary["% Passing Reading"] = spending_summary["% Passing Reading"].map("{:.2f}".format)
spending_summary["% Overall Passing"] = spending_summary["% Overall Passing"].map("{:.2f}".format)

spending_summary

,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
Spending Ranges (Per Student),,,,,
<$589,71.36,70.72,88.84,86.39,76.72
$585-630,72.07,71.03,91.52,87.29,79.88
$630-645,69.85,69.84,84.69,83.76,71.00
$645-600,68.88,69.05,81.57,81.77,66.76


## SCORES BY SCHOOL SIZE

In [9]:
# binning 'Total Students' into categories
bins = [0, 999, 1999, 4999]

# creating category names
budget_cat = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

# binning school_updated DataFrame made from previous section of homework
school_updated["Total Students"] = school_updated["Total Students"].astype(int)
school_updated["School Size"] = pd.cut(school_updated["Total Students"], 
                                                         bins, labels=budget_cat, include_lowest=True)

# Filtering school_updated so that scores_school only has relevant columns
size_summary = school_updated[["School Size", "Average Maths Score", "Average Reading Score",
                              "% Passing Maths", "% Passing Reading", "% Overall Passing"]]
size_summary = size_summary.groupby("School Size").mean()

# formatting
size_summary["Average Maths Score"] = size_summary["Average Maths Score"].map("{:.2f}".format)
size_summary["Average Reading Score"] = size_summary["Average Reading Score"].map("{:.2f}".format)
size_summary["% Passing Maths"] = size_summary["% Passing Maths"].map("{:.2f}".format)
size_summary["% Passing Reading"] = size_summary["% Passing Reading"].map("{:.2f}".format)
size_summary["% Overall Passing"] = size_summary["% Overall Passing"].map("{:.2f}".format)

size_summary

,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
School Size,,,,,
Small (<1000),72.34,71.64,90.81,87.56,79.07
Medium (1000-2000),71.42,70.72,89.85,86.71,78.04
Large (2000-5000),69.75,69.58,84.25,83.30,70.29


## SCORES BY SCHOOL SIZE

In [10]:
# Filtering school_updated so that scores_school only has relevant columns
type_summary = school_updated[["School Type", "Average Maths Score", "Average Reading Score",
                              "% Passing Maths", "% Passing Reading", "% Overall Passing"]]
type_summary = type_summary.groupby("School Type").mean()

# formatting
type_summary["Average Maths Score"] = type_summary["Average Maths Score"].map("{:.2f}".format)
type_summary["Average Reading Score"] = type_summary["Average Reading Score"].map("{:.2f}".format)
type_summary["% Passing Maths"] = type_summary["% Passing Maths"].map("{:.2f}".format)
type_summary["% Passing Reading"] = type_summary["% Passing Reading"].map("{:.2f}".format)
type_summary["% Overall Passing"] = type_summary["% Overall Passing"].map("{:.2f}".format)

type_summary

,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
School Type,,,,,
Government,69.83,69.68,84.46,83.59,70.70
Independent,71.37,70.72,89.20,86.25,76.97
